In [1]:
#@title Default title text
from ipywidgets import Layout, Button, Box, FloatText,Text, Password, Dropdown, Label, IntSlider,DatePicker
from IPython.display import clear_output
import datetime

if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/trupa/pnl-analysis
    %cd pnl-analysis/

api_key_in = Password()
api_secret_in = Password()
api_passphrase_in = Password()
api_group_in = Text()
start_date_in = DatePicker(disabled=False,value=datetime.datetime.now())
end_date_in = DatePicker(disabled=False,value=datetime.datetime.now())

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

op_item_passphrase_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

op_item_group_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

box_passPhrase = Box([Label(value='Passphrase'), 
    api_passphrase_in], layout=op_item_passphrase_layout)
box_group =   Box([Label(value='Api group'), 
    api_group_in], layout=op_item_group_layout)

form_items = [
    Box([Label(value='Api Key'), 
         api_key_in], layout=form_item_layout),
    Box([Label(value='Secret Key'), 
         api_secret_in], layout=form_item_layout),
    box_passPhrase,
    box_group,
    Box([Label(value='Start Date'), start_date_in], layout=form_item_layout),
    Box([Label(value='End Date'), end_date_in], layout=form_item_layout),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))

box_passPhrase.layout.display = 'none'
box_group.layout.display = 'flex'
display(form)

def get_exchange_data():

    import pandas as pd
    import numpy as np
    import json
    import plotly.graph_objects as go
    from IPython.core.display import display, HTML
    from datetime import datetime

    # Set display
    pd.options.display.float_format = '{:,.10f}'.format

    START_TIME=start_date_in.value
    END_TIME=end_date_in.value
    api_key = api_key_in.value
    api_secret = api_secret_in.value
    api_group = api_group_in.value
    client = None
    start_dt = int(pd.to_datetime(START_TIME).timestamp()*1000)
    end_dt = int(pd.to_datetime(END_TIME).timestamp()*1000)

    from src.ascendex.AscendexClientWrapper import AscendexClientWrapper
    client = AscendexClientWrapper.createInstance(api_key,api_secret,api_group)

    from jinja2 import Template
    
    class Printer:
        @staticmethod
        def h1(*title):
            title_t = Template("<h1>{{title}}</h1>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))
        @staticmethod
        def h2(*title):
            title_t = Template("<h2>{{title}}</h2>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))
        @staticmethod
        def h3(*title):
            title_t = Template("<h3>{{title}}</h3>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))

        @staticmethod
        def html(html):
            display(HTML(html))
        
        @staticmethod
        def p_df(df):
            """Neatly display a dataframe"""
            display(HTML(df.to_html()))
            
        def p_dict(dt):
            df = pd.DataFrame()
            df['_'] = dt.keys()
            df['__'] = dt.values()
            Printer.p_df(df)

    df_trades = client.get_all_trades(start_dt,end_dt)
    df_balances = client.get_current_asset_balances()

    return df_trades, df_balances


Cloning into 'pnl-analysis'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 282 (delta 136), reused 279 (delta 133), pack-reused 0
Receiving objects: 100% (282/282), 97.74 KiB | 12.22 MiB/s, done.
Resolving deltas: 100% (136/136), done.
/content/pnl-analysis


Box(children=(Box(children=(Label(value='Api Key'), Password()), layout=Layout(display='flex', flex_flow='row'…

In [ ]:
from src.processing import pnl_calculate
import pandas as pd
from IPython.core.display import display, HTML

df_trades_raw, df_balances_raw = get_exchange_data()

# Set display
pd.options.display.float_format = '{:,.10f}'.format

from jinja2 import Template
    
class Printer:
    @staticmethod
    def h1(*title):
        title_t = Template("<h1>{{title}}</h1>")
        display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))
    @staticmethod
    def h2(*title):
        title_t = Template("<h2>{{title}}</h2>")
        display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))
    @staticmethod
    def h3(*title):
        title_t = Template("<h3>{{title}}</h3>")
        display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))

    @staticmethod
    def html(html):
        display(HTML(html))
        
    @staticmethod
    def p_df(df):
        """Neatly display a dataframe"""
        display(HTML(df.to_html()))
            
    def p_dict(dt):
        df = pd.DataFrame()
        df['_'] = dt.keys()
        df['__'] = dt.values()
        Printer.p_df(df)

symbols = df_trades_raw.symbol.unique()

df_trades_collection = {}
df_balances_collection = {}
ticker_collection = {}

from src.ascendex.AscendexClientWrapper import AscendexClientWrapper
client = AscendexClientWrapper.createInstance('','','')

for s in symbols:
    base_tmp = s.split("/")[0]
    quote_tmp = s.split("/")[1]

    df_balances_collection[s] = df_balances_raw[(df_balances_raw['asset'] == base_tmp) | (df_balances_raw['asset'] == quote_tmp)]
    df_balances_collection[s].set_index("asset", inplace=True, drop=True)

    if df_balances_collection[s][df_balances_collection[s].index == base_tmp].empty:
        df_balances_collection[s].loc[base_tmp] = [0,0,0]
    
    df_trades_tmp = df_trades_raw[df_trades_raw['symbol'] == s]
    df_trades_collection[s] = df_trades_tmp[['price', 'qty', 'quoteQty', 'commission', 'commissionAsset', 'side', 'commissionAssetUsdPrice', 'date_time']]

    ticker_collection[s] = client.client.getTicker(symbol = s)

In [ ]:
total_fees = 0
total_pnl = 0
total_inventory_pnl = 0

for key in df_balances_collection:
    balance = df_balances_collection[key]
    ticker = ticker_collection[key]
    base = key.split("/")[0]
    quote = key.split("/")[1]
    base_price = balance.iloc[balance.index == base]['price'].squeeze()
    quote_price = balance[balance.index == quote]['price'].squeeze()
    df_trades = df_trades_collection[key]
    meta = {
        'base_asset':base,
        'quote_asset':quote,
        'quote_asset_price':quote_price,
        'base_asset_price':base_price
    }     

    Printer.h2(key)
    summary,df_summary_table,total_fees_usd,df_commissions = pnl_calculate(df_trades,balance,meta)

    Printer.h3(f"Fees: {df_summary_table[df_summary_table.index == 'Trading fees (usd value)']['Total'].squeeze()}")
    Printer.h3(f"Pnl: {df_summary_table[df_summary_table.index == 'Net pnl (USD)']['Total'].squeeze()}")
    Printer.h3(f"Inventory: {balance[balance.index == base]['balance'].squeeze()}, Open: {ticker['open']}, Close: {ticker['close']}")
    Printer.h3(f"Inventory Pnl: {balance[balance.index == base]['balance'].squeeze() * (float(ticker['open']) - float(ticker['close']))}")

    total_fees += df_summary_table[df_summary_table.index == 'Trading fees (usd value)']['Total'].squeeze()
    total_pnl += df_summary_table[df_summary_table.index == 'Net pnl (USD)']['Total'].squeeze()
    total_inventory_pnl += (balance[balance.index == base]['balance'].squeeze() * (float(ticker['open']) - float(ticker['close'])))

Printer.h2("Total summary")
Printer.h3(f"Total Fees: {total_fees}")
Printer.h3(f"Total Pnl: {total_pnl}")
Printer.h3(f"Total Inventory Pnl: {total_inventory_pnl}")


